In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pint import UnitRegistry

import scipy as sp
from scipy.optimize import fsolve

from fluid import Fluid

import json

with open('turbine_conditions.json') as f:
    conditions = json.load(f)



In [2]:
co2 = Fluid(fluid='co2')
m_out = 0.26 # kg/s
t_out = 150 + 273.15
delta_p = 2.1 * 1e6 # Pa

In [3]:
def solve_mass_flows(h1,h2,h_tot,m_tot):
    # TODO: fix where we strip
    # check that h_tot is between h1 and h2
    if not (h1 < h_tot < h2 or h2 < h_tot < h1):
        #print('h_tot is not between h1 and h2')
        return None
    
    # solve the linear system
    A = np.array([[1, 1], [h1 - h_tot, h2 - h_tot]])
    b = np.array([m_tot, 0])
    m1, m2 = np.linalg.solve(A, b)
    if m1 < 0 or m2 < 0:
        #print('mass flows are negative')
        return None
    
    return m1, m2

In [4]:
invalid_solution = []


for i, condition in enumerate(conditions):
    
    for flow_1 in range(1,7):
        
        enthalpy1 = co2.enthalpy(P = condition[f'P{flow_1}'], T = condition[f'T{flow_1}'])
        
        for flow_2 in range(flow_1 + 1,7):
            
            enthalpy2 = co2.enthalpy(P = condition[f'P{flow_2}'], T = condition[f'T{flow_2}'])
            
            for exhaust in range(1,7):
                # test if solution is valid
                if (flow_1, flow_2, exhaust) in invalid_solution:
                    continue
                
                
                if exhaust == flow_1 or exhaust == flow_2:
                    invalid_solution.append([flow_1, flow_2, exhaust])
                    continue
                p_tot = condition[f'P{exhaust}'] + delta_p
                
                
                # check to make sure the output pressure is lower than both the inputs
                if p_tot > condition[f'P{flow_1}'] or p_tot > condition[f'P{flow_2}']:
                    invalid_solution.append([flow_1, flow_2, exhaust])
                    continue
                
                
                h_tot = co2.enthalpy(P = p_tot, T = t_out)
                
                #print(f"m_out={m_out}, h_tot={h_tot}, e1={enthalpy1}, e2={enthalpy2}")
                
                # confirm units and make dimensionless
                m_tot = m_out
                h_tot = h_tot
                h1 = enthalpy1
                h2 = enthalpy2
                
                
                # check that the mass fractions are positive
                ms = solve_mass_flows(h1,h2,h_tot,m_tot)
                if ms is None:
                    invalid_solution.append([flow_1, flow_2, exhaust])
                    continue
                m1, m2 = ms
                #print(f"{i} - {flow_1}, {flow_2}, {exhaust}:  {m1}, {m2}")
    
    
    
    

In [5]:
valid_combinations = []

for condition in conditions:
    
    for flow_1 in range(1, 7):
        for flow_2 in range(flow_1 + 1, 7):
            for exhaust in range(1, 7):
                if [flow_1, flow_2, exhaust] in invalid_solution or [flow_1, flow_2, exhaust] in valid_combinations:
                    continue
                
                valid_combinations.append([flow_1, flow_2, exhaust])
    
print("Valid Combinations Include: ")
print(valid_combinations)


Valid Combinations Include: 
[[2, 4, 1], [2, 4, 5], [2, 4, 6]]


In [6]:
def Cv(p_in, s_in, p_out_ideal,
       mdot, # mass flow rate
       Cd=0.61 # assuming a sharp edged orifice
       ):
    s_out = s_in
    # try real gas first
    def v_out(p_out):
        h_in = co2.enthalpy(P = p_in, S = s_in)
        h_out = co2.enthalpy(P = p_out, S = s_out)
        v_in = 0 # assuming input velocity is negligible
        return np.sqrt(2 * (h_in - h_out))
    
    c = lambda p_out: co2.sound_speed(P = p_out, S = s_out)
    
    if v_out(p_out_ideal) > c(p_out_ideal):
        p_out_real = fsolve(lambda p_out: v_out(p_out)-c(p_out),x0=p_out_ideal)
    else: # otherwise solve for choked flow
        p_out_real = p_out_ideal
    
    # find the area of the orifice
    A2 = mdot / co2.density(P = p_out_real, S = s_out) / v_out(p_out_real)

    valve_Cv = Cd * 58888.5 * A2
    return valve_Cv, A2

In [1]:
output = 5 # choosing 5 because it's already at the right temperature

min_Cv2, max_Cv2 = None, None
min_Cv4, max_Cv4 = None, None


for condition in conditions:
    enthalpy2 = co2.enthalpy(P = condition['P2'], T = condition['T2'])
    enthalpy4 = co2.enthalpy(P = condition['P4'], T = condition['T4'])
    p_out = condition[f'P{output}'] + delta_p
    enthalpy_tot = co2.enthalpy(P = p_out, T = condition[f'T{output}'])

    m2, m4 = solve_mass_flows(h1=enthalpy2,
                              h2=enthalpy4,
                              h_tot=enthalpy_tot,
                              m_tot=m_out)
    
    
    
    ###
    # pull out state variables
    p2 = condition['P2']
    p4 = condition['P4']
    s2 = co2.entropy(P = condition['P2'], T = condition['T2'])
    s4 = co2.entropy(P = condition['P4'], T = condition['T4'])

    Cv2 = Cv(p_in=p2, s_in=s2, p_out_ideal=p_out, mdot=m2)
    Cv4 = Cv(p_in=p4, s_in=s4, p_out_ideal=p_out, mdot=m4)

    if min_Cv2 is None or Cv2 < min_Cv2:
        min_Cv2 = Cv2
    if max_Cv2 is None or Cv2 > max_Cv2:
        max_Cv2 = Cv2

    if min_Cv4 is None or Cv4 < min_Cv4:
        min_Cv4 = Cv4
    if max_Cv4 is None or Cv4 > max_Cv4:
        max_Cv4 = Cv4
        
print(f"Min and Max Cv for valve 2: {min_Cv2}, {max_Cv2}")
print(f"Min and Max Cv for valve 4: {min_Cv4}, {max_Cv4}")

NameError: name 'conditions' is not defined